In [54]:
import pandas as pd
import numpy as np 

df_raw = pd.read_csv("WA_Fn-UseC_-Telco-Customer-Churn.csv")


In [55]:
df_raw.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [56]:
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   gender            7043 non-null   object 
 2   SeniorCitizen     7043 non-null   int64  
 3   Partner           7043 non-null   object 
 4   Dependents        7043 non-null   object 
 5   tenure            7043 non-null   int64  
 6   PhoneService      7043 non-null   object 
 7   MultipleLines     7043 non-null   object 
 8   InternetService   7043 non-null   object 
 9   OnlineSecurity    7043 non-null   object 
 10  OnlineBackup      7043 non-null   object 
 11  DeviceProtection  7043 non-null   object 
 12  TechSupport       7043 non-null   object 
 13  StreamingTV       7043 non-null   object 
 14  StreamingMovies   7043 non-null   object 
 15  Contract          7043 non-null   object 
 16  PaperlessBilling  7043 non-null   object 


In [57]:
df_raw.shape

(7043, 21)

In [58]:
df_raw['TotalCharges'] = pd.to_numeric(df_raw['TotalCharges'],errors='coerce')

In [59]:
df_raw['customerID'].is_unique

True

In [60]:
df_raw.isna().sum()

customerID           0
gender               0
SeniorCitizen        0
Partner              0
Dependents           0
tenure               0
PhoneService         0
MultipleLines        0
InternetService      0
OnlineSecurity       0
OnlineBackup         0
DeviceProtection     0
TechSupport          0
StreamingTV          0
StreamingMovies      0
Contract             0
PaperlessBilling     0
PaymentMethod        0
MonthlyCharges       0
TotalCharges        11
Churn                0
dtype: int64

In [61]:
df = df_raw.copy()

In [62]:
df.columns = df.columns.str.lower()

In [63]:
df['tenure_group'] = pd.cut(df['tenure'],
                            bins=[0,12,24,36,48,60,100],
                            labels=['0-12','12-24','24-36','36-48','48-60','60+'])

In [64]:
df['monthly_charges'] = df['monthlycharges']

In [65]:
df.to_csv("telco_churn_clean.csv",index=False)

In [66]:
dim_customer = df[[
    'customerid','gender','seniorcitizen','partner','dependents','tenure','tenure_group'
]].drop_duplicates()

In [67]:
dim_customer.rename(columns={
    'customerid':'customer_id',
    'seniorcitizen':'senior_citizen'
},inplace=True)

In [68]:
dim_customer.to_csv("dim_customer.csv",index=False)

In [69]:
dim_subscription = df[[
    'customerid','contract','paymentmethod','paperlessbilling','monthly_charges','internetservice'
]].drop_duplicates()

In [70]:
dim_subscription.rename(columns={
    'customerid':'customer_id',
    'paymentmethod':'payment_method',
    'paperlessbilling':'paperless_billing',
    'internetservice':'internet_service'
}, inplace=True)

In [71]:
dim_subscription.to_csv("dim_subscription.csv",index=False)

In [72]:
fact_rows=[]

for _, row in df.iterrows():
    for month in range(1,row['tenure'] +1):
        fact_rows.append({
            'customer_id': row['customerid'],
            'month_number': month,
            'monthly_revenue': row['monthly_charges'],
            'churn_flag': 1 if (row['churn'] =='Yes' and month==row['tenure']) else 0
        })

fact_customer_monthly = pd.DataFrame(fact_rows)

In [73]:
fact_customer_monthly['engagement_score'] = np.where(
    fact_customer_monthly['monthly_revenue'] > 70,'High',
    np.where(fact_customer_monthly['monthly_revenue'] > 40, 'Medium', 'Low')
)

In [74]:
fact_customer_monthly.to_csv("fact_customer_monthly.csv", index=False)

In [75]:
customer_features = (fact_customer_monthly.groupby('customer_id')
                     .agg(total_revenue=('monthly_revenue','sum'),
                          avg_monthly_revenue=('monthly_revenue','mean'),
                          months_active=('month_number','max'),
                          churned=('churn_flag','max')
                         ).reset_index())

In [76]:
customer_features = customer_features.merge(dim_subscription[['customer_id','contract']],
                                            on='customer_id',how='left')


In [77]:
customer_features = customer_features.merge(fact_customer_monthly
                                            .groupby('customer_id')['engagement_score']
                                            .agg(lambda x: x.value_counts().idxmax())
                                            .reset_index(),on='customer_id',how='left')


In [78]:
segmentation_data = customer_features[['total_revenue','avg_monthly_revenue','months_active']]

In [79]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaled_data = scaler.fit_transform(segmentation_data)

In [80]:
from sklearn.cluster import KMeans

inertia = []
for k in range(2,8):
    km = KMeans(n_clusters=k, random_state=42)
    km.fit(scaled_data)
    inertia.append(km.inertia_)

In [81]:
kmeans=KMeans(n_clusters=4,random_state=42)
customer_features['segment']=kmeans.fit_predict(scaled_data)

In [82]:
segment_map= {
    0: "Champions",
    1: "At-Risk Loyalists",
    2: "Price-Sensitive",
    3: "New & Unstable"
}
customer_features['segment_name'] = customer_features['segment'].map(segment_map)
customer_features['segment_name'] = customer_features['segment_name'].fillna("Unclassified")

In [83]:
segment_profile = (customer_features.groupby(['segment','segment_name'])
                   .agg(customers=('customer_id','count'),
                        avg_total_revenue=('total_revenue','mean'),
                        avg_months_active=('months_active','mean'),
                        churn_rate=('churned','mean')).reset_index())

In [84]:
customer_features.to_csv("customer_features.csv",index = False)

In [85]:
segment_profile.to_csv("segment_profile.csv", index = False)

In [86]:
customer_features['segment'].value_counts()


segment
1    2282
3    1906
0    1690
2    1154
Name: count, dtype: int64